#データ準備

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pwd
%cd /content/drive/My Drive/Colab Notebooks/NLP20211H

/content
/content/drive/My Drive/Colab Notebooks/NLP20211H


In [3]:
%matplotlib inline

In [4]:
!pip install mecab-python3
!pip install unidic-lite
import MeCab

     |████████████████████████████████| 491kB 2.9MB/s 
     |████████████████████████████████| 47.4MB 63kB/s 
  Created wheel for unidic-lite: filename=unidic_lite-1.0.8-cp37-none-any.whl size=47658838 sha256=342fe7ae3deb98c74a05036c66e3542558fa7b729278dfb6250a182a158d8351
  Stored in directory: /root/.cache/pip/wheels/20/48/8d/b66d8361a27f58f41ec86640e4fd2640de0403a6367511eab7
Successfully built unidic-lite


In [5]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

SOS_token = 0 #文頭マークを予約
EOS_token = 1 #文末マークを予約

class Lang:
    #+
    #言語ごとに、単語からIndex,Indexから単語へ変換する辞書を準備
    #-
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

#+
#英語に関しては、全小文字にして、その他特殊文字を無視するように正規化
#-
def normalizeString(s):
    s = s.lower().strip()
    s = re.sub(r"([.!?])", r" \1", s) #rは文字列を生で扱うraw演算子(例：\を\として扱う]) 、re.subrは正規表現置き換えメソッド
    s = re.sub(r"[^a-zA-Z0-9.!?]+", r" ", s) #アルファベット、数字以外の特殊文字を空白に置き換え
    return s

def readLangs(lang1,lang2,reverse=False):  #第3引数は省略されたらFalse
    print("Reading lines...")
    tagger = MeCab.Tagger("-Owakati")
    lines = open('jpn.txt', encoding='utf-8').read().strip().split('\n') #jpn.txtに、英語、日本語のペアが入っている
    # 英語文は文字を正規化、日本語文は分かち書きして、ペアを作る
    pairs = [
             [ normalizeString(l.split('\t')[0]), (tagger.parse(l.split('\t')[1])).replace('\n','') ] 
             for l in lines
             ]
    print(pairs[:5])

    # ソースとターゲットを逆にしたいときのため
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)
    return input_lang, output_lang, pairs

#+
#データが多いと語彙数が増えて負荷が大きいので、10単語以下で、かつ以下のパターンを持つ英文を含む行だけにフィルターする
#-
MAX_LENGTH = 10
eng_prefixes = ("i am ", "i m ", "he is", "he s ", "she is", "she s ", "you are", "you re ", "we are", "we re ",  "they are", "they re ")
def filterPairs(pairs, reverse): 
    def filterPair(p, reverse):
      if reverse:
        return (len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH) and p[1].startswith(eng_prefixes)
      else:
        return (len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH) and p[0].startswith(eng_prefixes)
    return [pair for pair in pairs if filterPair(pair, reverse)]

#+
#ファイルからデータを読み込み、フィルターし、input_lang, output_langにそれぞれ格納する
#-
def prepareData(lang1, lang2, reverse=False): #第3引数は省略されたらFalse
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs, reverse)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

input_lang, output_lang, pairs = prepareData('eng', 'jpn', False)
print(random.choice(pairs))
print(pairs[:5])

Reading lines...
[['go .', '行け 。 '], ['go .', '行き なさい 。 '], ['hi .', 'こんにちは 。 '], ['hi .', 'もしもし 。 '], ['hi .', 'やっほー 。 ']]
Read 48955 sentence pairs
Trimmed to 2080 sentence pairs
Counting words...
Counted words:
eng 1230
jpn 1560
['i m very happy to see you .', 'お 会い でき て とても 嬉しい です 。 ']
[['i m 19 .', '１９ 歳 です 。 '], ['i m ok .', '大丈夫 です よ 。 '], ['i m ok .', '私 は 大丈夫 です 。 '], ['i m up .', '起き てる よ 。 '], ['i m tom .', 'トム と 申し ます 。 ']]


#Seq2Seq

In [6]:
#+
#入力文をGRUで符号化する
#-
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)  #単語の埋め込み表を作る
        self.gru = nn.GRU(hidden_size, hidden_size) #GRUオブジェクトを生成する

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1) #入力単語の埋め込みベクトルを取り出す
        output, hidden = self.gru(embedded, hidden) #GRUに通す
        return output, hidden #新しい隠れ状態を返す

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

#+
#入力文をGRUで符号化した結果（最終隠れ状態）を使って、出力単語列に復元する
#-
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(output_size, hidden_size) #埋め込み表を準備する
        self.gru = nn.GRU(hidden_size, hidden_size) #GRUオブジェクトを作る
        self.out = nn.Linear(hidden_size, output_size) #GRUの出力を出力言語語彙数サイズのベクトルへ全結合
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = F.relu(self.embedding(input).view(1, 1, -1)) #単語の埋め込みベクトルを得る
        output, hidden = self.gru(output, hidden) #GRUに通して、出力（次単語予測）を得る
        output = self.softmax(self.out(output[0])) #全結合で出力言語語彙数サイズのベクトルへ変換し、softmax
        return output, hidden #次単語予測（次回は入力となる）と、隠れ状態を返す

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [7]:
#+
#文を単語IDリストにする
#-
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

#-
#文を、単語の並びのテンソルにする
#-
def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token) #文末マークを付加する
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1) #(単語数,1)のshapeにしてテンソル化

#+
#入力文とターゲット文をそれぞれ、単語を並べたテンソルにする
#-
def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0]) #入力文をshape(単語数,1)の行列にする  
    target_tensor = tensorFromSentence(output_lang, pair[1]) #ターゲット文をshape(単語数,1)の行列にする 
    return (input_tensor, target_tensor)

def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden() #エンコーダーの隠れ状態を初期化
    encoder_optimizer.zero_grad() #エンコーダーの勾配を初期化
    decoder_optimizer.zero_grad() #デコーダーの勾配を初期化
    input_length = input_tensor.size(0) #入力単語数をとる
    target_length = target_tensor.size(0) #ターゲット単語数をとる
    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device) #エンコーダーの出力を初期化
    loss = 0

    for ei in range(input_length): #入力単語数だけエンコーダーを回す
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden) #エンコーダーに入力単語を1個づつ渡す
        #エンコーダーのouputは利用していないことに注意

    decoder_input = torch.tensor([[SOS_token]], device=device) #デコーダーの最初の入力を文頭マークにする
    decoder_hidden = encoder_hidden #エンコーダーの最終隠れ状態を、デコーダーの隠れ状態の初期値にする
    for di in range(target_length): #デコーダーを出力単語数だけ回す
        decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden) #デコーダーに分党から始まる単語を1個づつ予測させる
        loss += criterion(decoder_output, target_tensor[di]) #正解単語と比べてロスを計算する
        topv, topi = decoder_output.topk(1) #予測単語を得る
        decoder_input = topi.squeeze().detach()  #予測単語を取り出して、次のループの入力に設定する
        if decoder_input.item() == EOS_token: break #予測単語が文末ならループを出る
   
    loss.backward() #逆伝播
    encoder_optimizer.step() #学習レートを調整
    decoder_optimizer.step() #学習レートを調整
    return loss.item() / target_length #平均ロスを返す

def trainIters(encoder, decoder, n_iters, every=1000):
    loss_total = 0  # Reset every print_every
    training_pairs = [tensorsFromPair(random.choice(pairs)) for i in range(n_iters)] #トレーニングデータを取り出す
    criterion = nn.NLLLoss() #ロスをクロスエントロピーに設定

    for i in range(1, n_iters + 1): #n_itersだけトレーニングを回す
        training_pair = training_pairs[i - 1]
        input_tensor = training_pair[0] #入力文の単語列、shape(単語数,1)
        target_tensor = training_pair[1] #ターゲット文の単語列、shape(単語数,1)
        loss = train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        loss_total += loss
        if i % every == 0:
            loss_avg = loss_total / every
            loss_total = 0
            print('(%d %d%%) %.4f' % (i, i / n_iters * 100, loss_avg))

hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device) #エンコーダーオブジェクト作成
decoder1 = DecoderRNN(hidden_size, output_lang.n_words).to(device) #デコーダーオブジェクト作成
learning_rate=0.01
encoder_optimizer = optim.SGD(encoder1.parameters(), lr=learning_rate) #SGDオプティマイザーオブジェクト作成
decoder_optimizer = optim.SGD(decoder1.parameters(), lr=learning_rate) #SGDオプティマイザーオブジェクト作成
trainIters(encoder1, decoder1, 50000) #trainItersを起動
#トレーニングに時間がかかるので終了したらいったん保存。
torch.save({ 
            'enc': encoder1.state_dict(),
            'dec': decoder1.state_dict(),
            'eopt': encoder_optimizer.state_dict(),
            'dopt': decoder_optimizer.state_dict(),
            }, 'seq2seq.pt')

(1000 2%) 3.6985
(2000 4%) 3.3566
(3000 6%) 3.1957
(4000 8%) 3.0646
(5000 10%) 2.9030
(6000 12%) 2.7358
(7000 14%) 2.5682
(8000 16%) 2.3587
(9000 18%) 2.3117
(10000 20%) 2.0308
(11000 22%) 2.0047
(12000 24%) 1.7912
(13000 26%) 1.6514
(14000 28%) 1.5485
(15000 30%) 1.4971
(16000 32%) 1.3049
(17000 34%) 1.2549
(18000 36%) 1.1474
(19000 38%) 1.0425
(20000 40%) 0.9922
(21000 42%) 0.9120
(22000 44%) 0.8568
(23000 46%) 0.7808
(24000 48%) 0.7273
(25000 50%) 0.7648
(26000 52%) 0.6387
(27000 54%) 0.6254
(28000 56%) 0.5863
(29000 57%) 0.5789
(30000 60%) 0.5140
(31000 62%) 0.5165
(32000 64%) 0.4918
(33000 66%) 0.4629
(34000 68%) 0.4440
(35000 70%) 0.4510
(36000 72%) 0.4311
(37000 74%) 0.3950
(38000 76%) 0.4156
(39000 78%) 0.3766
(40000 80%) 0.3524
(41000 82%) 0.3452
(42000 84%) 0.3386
(43000 86%) 0.3591
(44000 88%) 0.3706
(45000 90%) 0.3607
(46000 92%) 0.3633
(47000 94%) 0.3089
(48000 96%) 0.3350
(49000 98%) 0.3458
(50000 100%) 0.2995


In [8]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad(): #勾配計算しないモード
        input_tensor = tensorFromSentence(input_lang, sentence) #入力文をID列のテンソルにする
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden() #エンコーダーの隠れ状態初期化
        for ei in range(input_length): #入力単語数だけ、ループを回す
            encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden) #エンコーダーで隠れ状態を更新
        decoder_input = torch.tensor([[SOS_token]], device=device)  # デコーダーの最初の入力に文頭マークを設定
        decoder_hidden = encoder_hidden #エンコーダーの出力をデコーダーの隠れ状態の初期値とする
        decoded_words = []
        for di in range(max_length): #文末を予測するまで、ループ
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden) #デコーダーで次の単語を予測し、隠れ状態更新
            topv, topi = decoder_output.data.topk(1) #次に来る予測単語を取り出す
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>') #文末を予測したら終了
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()]) #予測単語を出力単語リストに追加
            decoder_input = topi.squeeze().detach() #デコーダへの入力として、今得た予測単語を設定
        return decoded_words
  
def evaluateRandomly(encoder, decoder, n=10): #ランダムに10個翻訳させる
    for i in range(n):
        pair = random.choice(pairs) 
        print('>', pair[0])
        print('=', pair[1])
        output_words = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder1 = DecoderRNN(hidden_size, output_lang.n_words).to(device)
learning_rate=0.01
encoder_optimizer = optim.SGD(encoder1.parameters(), lr=learning_rate)
decoder_optimizer = optim.SGD(decoder1.parameters(), lr=learning_rate)
#エンコーダー、デコーダーのパラメータを保存ファイルからロード
m = torch.load('seq2seq.pt')
encoder1.load_state_dict(m['enc'])
decoder1.load_state_dict(m['dec'])
encoder_optimizer.load_state_dict(m['eopt'])
decoder_optimizer.load_state_dict(m['dopt'])
evaluateRandomly(encoder1, decoder1,30)

> he is a fast runner .
= 彼 は 走る の が 速い 。 
< 彼 は 走る の が 速い 。  <EOS>

> he is busy with job hunting .
= 彼 は 職探し に 忙しい 。 
< 彼 は 職探し に 忙しい 。  <EOS>

> i m breast feeding my baby .
= 母乳 で 育て て い ます 。 
< 母乳 で 育て て い ます 。  <EOS>

> he is an honest man .
= 彼 は 正直 な 男 だ 。 
< 彼 は 正直 な 男 だ 。  <EOS>

> i m sorry for being late .
= 遅く なっ て 申し訳 あり ませ ん 。 
< 遅れ なっ ごめん 。  。  <EOS>

> he is looking for a job .
= 彼 は 職 を 探し て いる 。 
< 彼 は 職 を 探し て いる 。  <EOS>

> i m leaving town for a few days .
= 数 日 町 を 離れ ます 。 
< 数 日 町 を 離れ ます 。  <EOS>

> we are going to have a storm .
= 嵐 に なる だろう 。 
< 嵐 に なる だろう 。  <EOS>

> she is very clever .
= 彼女 は とても 利口 だ 。 
< 彼女 は とても 賢い 人 だ 。  <EOS>

> she is a little shy .
= あの 子 ちょっと シャイ な の 。 
< あの 子 ちょっと シャイ な の 。  <EOS>

> she is attractive .
= 彼女 は 魅力 的 だ 。 
< 彼女 は 魅力 的 だ 。  <EOS>

> you re just guessing .
= 単なる 推測 だろ 。 
< 単なる 推測 だろ 。  <EOS>

> i m cooperative .
= 私 は 協調 性 が ある 。 
< 私 は 協調 性 が ある 。  <EOS>

> she is now on vacation .
= 彼女 は 休暇 中 です 。 
< 彼女 は 休暇 中 です 。  